# Use a locally trained model

Congratulations, you just trained your own model. We'll assume that we have a model folder named 'my_model-1.0' located in '/work'. (See the previous tutorial on how to [train your models](./3.%20Advanced%20usage%20-%20Training%20a%20model%20on%20the%20UD%20Corpus.ipynb) to put things into context)

This notebook looks at how to train a model using the Universal Dependencies Corpus. 
We will learn how to (1) download the UD Corpus, (2) train a tokenizer and a tagger model on a specific language and then (3) pack it all up in a zip model that we'll use locally.

This notebook is run on a 18.04 Ubuntu, with Python3 installed. Assume we are working in folder ``/work``. Also, let's assume that NLP-Cube is installed locally in ``/work/NLP-Cube``. If you do not have NLP-Cube installed locally (not using pip install, but direclty cloning the github repo), please first follow the [local install guide](./2.%20Advanced%20usage%20-%20NLP-Cube%20local%20installation.ipynb).


### Using NLP-Cube from your own project

Let's create a "project" named ``my_project`` with a single ``main.py`` file, and place it in ``/work``.

The folder should look like:

In [1]:
! ls /work/my_project

main.py


The file has the following code:

```
import sys, os

root_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
nlp_cube_path = os.path.join(root_path,"NLP-Cube")
os.sys.path.insert(0, nlp_cube_path)

my_model_root = "/work" 
local_embeddings_file = os.path.join("/work","wiki.en.vec")

from cube.api import Cube
cube=Cube(verbose=True)
cube.load(language_code="my_model", version="1.0", local_models_repository=my_model_root, local_embeddings_file=local_embeddings_file)

text = "I'm a success today because I had a friend who believed in me and I didn't have the heart to let him down. This is a quote by Abraham Lincoln."
sentences = cube(text)
for sentence in sentences:
    print()
    for token in sentence:
        line = ""
        line += str(token.index) + "\t"
        line += token.word + "\t"
        line += token.lemma + "\t"
        line += token.upos + "\t"
        line += token.xpos + "\t"
        line += token.attrs + "\t"
        line += str(token.head) + "\t"
        line += token.label + "\t"
        line += token.deps + "\t"
        line += token.space_after
        print(line)
```

Running it will print:

In [22]:
! cd /work/my_project && python3 main.py

[dynet] random seed: 3654745557
[dynet] allocating memory: 512MB
[dynet] memory allocation done.
	Loading embeddings... 
	Loading tokenization model ...
	Lemmatizer is not available on this model. 
	Loading tagger model ...
	Parsing is not available on this model... 
Model loading complete.


1	I	_	PRON	PRP	Case=Nom|Number=Sing|Person=1|PronType=Prs	0	_	_	SpaceAfter=No
2	'm	_	AUX	VBP	Mood=Ind|Tense=Pres|VerbForm=Fin	0	_	_	_
3	a	_	DET	DT	Definite=Ind|PronType=Art	0	_	_	_
4	success	_	NOUN	NN	Number=Sing	0	_	_	_
5	today	_	NOUN	NN	Number=Sing	0	_	_	_
6	because	_	SCONJ	IN	_	0	_	_	_
7	I	_	PRON	PRP	Case=Nom|Number=Sing|Person=1|PronType=Prs	0	_	_	_
8	had	_	VERB	VBD	Mood=Ind|Tense=Past|VerbForm=Fin	0	_	_	_
9	a	_	DET	DT	Definite=Ind|PronType=Art	0	_	_	_
10	friend	_	NOUN	NN	Number=Sing	0	_	_	_
11	who	_	PRON	WP	PronType=Rel	0	_	_	_
12	believed	_	VERB	VBD	Mood=Ind|Tense=Past|VerbForm=Fin	0	_	_	_
13	in	_	ADP	IN	_	0	_	_	_
14	me	_	PRON	PRP	Case=Acc|Number=Sing|Person=1|PronType=Prs	0	_	_	_
15	and	_	C

That's it. You just tokenized and tagged your own set of sentences.

---

#### A few details

Below we'll analyze each line of the script:

We start with:
```
import sys, os

root_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
nlp_cube_path = os.path.join(root_path,"NLP-Cube")
os.sys.path.insert(0, nlp_cube_path)
```
where we manually insert the path to NLP-Cube. Remember that ``NLP-Cube``, ``my_project`` and ``my_model-1.0`` are all located in ``/work``. Simply compose the path from your script to NLP-Cube, or, use any other method for your script to have access to NLP-Cube's libraries (like extending NLP-Cube or including it in your own project as a package). 

```
my_model_root = "/work" 
local_embeddings_file = os.path.join("/work","wiki.en.vec")
```
NLP-Cube's model library is structured as a "root" folder with subfolders as models. This is why we need to specify ``/work`` as the model root folder, and let NLP-Cube then select ``my_model-1.0`` as the model we want. Also, for locally trained models we need to specify a path to our vector embeddings file (because we don't have a metadata.json file yet - we'll see in the next [tutorial](./5.%20Advanced%20usage%20-%20Model%20export%20and%20import.ipynb) how to do that). Our embeddings file is simply ``/work/wiki.en.vec``.

```
from cube.api import Cube
cube=Cube(verbose=True)
cube.load(language_code="my_model", version="1.0", local_models_repository=my_model_root, local_embeddings_file=local_embeddings_file)
```

we import the Cube object, initialize it, and load our model. Notice that we specify which model we want by name directly in ``language_code="my_model"`` and its version in the ``version="1.0"``. If we don't specify a version, it will automatically pick the latest (biggest number) version. 

Because we trained a local model, we need to specify the local model repository, in this case simply ``/work``, and the full path to the local embeddings file.

Next, we calling ``sentences = cube(text)`` will automatically process the text.

### What about if I don't want to use embeddings?

Sometimes less is more. NLP-Cube uses several concurrent input encodings, including external embeddings. Sometimes these embeddings add less value than the other internal encodings. Plus, vector embeddings are usually large files. If you don't want to train a model using embeddings, you need to create a **dummy** embeddings file. 

For example, either use the ``examples/wiki.dummy.vec`` file, or create a "wiki.dummy.vec" (or any name you wish) that contains the following lines (copy-paste this in a new file):

```
1 300
</s> -0.00291 -0.18625 0.058028 -0.29661 0.068632 0.28946 0.0051565 0.17799 0.13018 -0.19517 -0.19083 0.0064972 -0.10813 -0.24357 -0.37569 -0.073037 0.16525 0.096274 -0.21967 -0.060706 -0.10286 -0.084604 0.1744 -0.031249 -0.10277 0.042256 0.20297 -0.039783 -0.028972 0.19946 -0.0046892 0.28466 -0.069435 0.068676 -0.023165 -0.046733 0.018789 -0.22607 -0.21216 0.17036 0.17399 0.066895 0.10422 0.15653 0.18378 -0.11769 0.1509 -0.081692 0.23579 -0.0093485 0.15167 -0.0097952 -0.035584 -0.0023132 0.10254 0.10207 -0.28487 -0.14675 -0.073192 0.056664 -0.019519 0.088953 0.079022 0.022475 -0.27715 0.08987 -0.038999 0.0028215 0.096766 0.32482 -0.18077 -0.15867 0.042736 -0.098426 0.19944 -0.31784 -0.1702 0.069564 -0.13843 -0.002799 0.3482 -0.011427 0.063471 -0.067671 0.013669 0.29324 0.089274 0.21365 0.27761 -0.31488 0.093548 0.020789 -0.020138 0.1133 0.14776 0.42006 -0.29999 -0.051176 0.11858 0.032608 0.33327 -0.19025 0.059251 -0.033972 -0.11209 0.11292 0.042999 0.23898 -0.056097 -0.052971 0.22852 0.050305 -0.012704 -0.081334 -0.027748 0.3725 -0.13769 0.17957 -0.094775 0.029658 -0.025596 0.32383 -0.039333 -0.2727 -0.29954 0.045625 0.11779 -0.29941 0.15999 -0.068357 0.083792 0.1545 -0.077238 0.15015 0.016603 -0.035233 0.16562 -0.19231 -0.37249 0.083253 -0.1229 -0.12096 0.33754 0.29348 0.21091 -0.3435 0.13705 -0.065175 -0.29146 -0.041733 -0.28036 0.16005 0.0086172 -0.020325 0.012445 -0.15517 0.20095 -0.0010922 0.23908 0.27801 0.13009 -0.1019 -0.059306 0.15052 -0.049257 0.10735 0.24883 -0.035315 -0.015704 0.1297 -0.26409 -0.17914 -0.34641 0.19633 0.12695 0.20356 0.32595 -0.013281 0.068872 -0.063561 -0.076011 0.066515 -0.18736 -0.058394 -0.11234 0.17197 0.24167 0.11705 0.012847 0.040238 0.092364 0.33407 -0.1206 -0.074965 0.22935 -0.035572 0.10584 -0.097787 -0.063045 -0.13527 0.053755 -0.33137 -0.051164 0.02706 0.059661 -0.32057 0.3829 0.1358 -0.086782 0.11528 -0.23391 0.21434 -0.12766 0.059699 -0.25511 -0.039314 -0.12894 -0.012722 0.2139 0.10244 -0.21011 -0.21161 -0.012924 0.19177 0.04161 0.084953 -0.06967 0.066996 -0.058172 0.25607 -0.2864 0.0041426 -0.38308 -0.021462 -0.17859 -0.32166 -0.029291 0.11121 0.18469 0.16992 -0.015047 -0.2933 -0.28637 0.2433 -0.042533 0.20242 -0.1547 -0.31574 -0.13264 0.11957 -0.37728 0.019524 -0.2068 0.083229 -0.12357 0.048097 -0.41851 0.55805 -0.024595 -0.15514 -0.0063529 0.21332 0.33929 0.32646 -0.079572 0.34776 -0.077371 0.13704 -0.38439 -0.0092563 0.055568 0.048388 0.081304 0.17917 0.054183 -0.072919 0.12038 -0.14533 0.25047 0.32786 0.31443 -0.16633 0.19101 0.069506 0.27939 -0.078308 0.1836 -0.07276 -0.057231 0.017672 -0.083664 0.11733 -0.0621 -0.0044557 -0.31261 0.069088 0.13869 -0.072683 -0.11379 0.037591  

```
It has to be a valid embedding file that only contains an unused End Of Sentence symbol. Thus, for NLP-Cube this will always look like an empty embeddings file. Then use this file whenever there is an ``--embedding`` parameter required, and also when you train your models with it.


---

In the following tutorial we'll cover how to [export your model for others to use](./5.%20Advanced%20usage%20-%20Model%20export%20and%20import.ipynb) in their own NLP-Cube installation. 
